In [1]:
!pip install -q langchain_community tiktoken langchain-openai langchainhub chromadb langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.2 MB/s eta 0:00:0

In [ ]:
import os
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]="<API_KEY>"
os.environ["LANGCHAIN_PROJECT"]="RAG_ADVANCED"
os.environ['OPENAI_API_KEY'] = "<API_KEY>"


from langsmith import utils
utils.tracing_is_enabled()

True

In [4]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [5]:
#Load Documents
loader = WebBaseLoader(
    web_paths = ("https://lilianweng.github.io/posts/2023-06-23-agent/", ),
    bs_kwargs = dict(
        parse_only = bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [12]:
#Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 300, chunk_overlap=50)
splits = text_splitter.split_documents(docs)

#Embed
embedding = OpenAIEmbeddings(model = "text-embedding-3-small")

vectorestore = Chroma.from_documents(documents=splits, embedding = embedding, persist_directory="./db002")

#Retriever
retriever = vectorestore.as_retriever()

In [19]:

from langchain.prompts import ChatPromptTemplate

#Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five different versions of the given user question to retrieve relavant documents from a vector database.
By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search.
Provide these alternative questions seperated by newlines. Original question:{question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = "gpt-4o-mini", temperature=0)

generate_queries = (
    prompt_perspectives
    | llm
    | StrOutputParser()
    | (lambda x:x.split("\n"))
)

In [24]:
from langchain.load import dumps, loads

def get_unique_union(documents:list[list]):
  """Unique union of retrieved docs"""
  #Flatten list of lists, and convert each Document to string
  flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
  #get unique documents
  unique_docs = list(set(flattened_docs))
  #Return
  return [loads(doc) for doc in unique_docs]

#Retrieve
question = "What is task decomposition for LLM agents?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

10

In [25]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

#RAG
template = """Answer the following question based on this context:

{context}

Question:{question}
"""

prompt = ChatPromptTemplate.from_template(template)
llm = ChatOpenAI(model = "gpt-4o-mini", temperature=0)

final_rag_chain = (
    {"context":retrieval_chain,
    "question":itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'Task decomposition for LLM (large language model) agents involves breaking down complex tasks into smaller, manageable subgoals. This process enables the agent to handle intricate tasks more efficiently. Task decomposition can be achieved in several ways:\n\n1. **Simple Prompting**: Using prompts like "Steps for XYZ." or "What are the subgoals for achieving XYZ?" to guide the LLM in identifying the necessary steps.\n2. **Task-Specific Instructions**: Providing specific instructions tailored to the task, such as "Write a story outline." for writing a novel.\n3. **Human Inputs**: Involving human guidance to assist in the decomposition process.\n\nBy effectively decomposing tasks, LLM agents can plan and execute actions more systematically, although challenges remain in long-term planning and adapting to unexpected errors.'